In [1]:
import sys
from unified_planning.shortcuts import *

In [2]:
Location = UserType('Location')
Robot = UserType('Robot')

at = Fluent('at', BoolType(), robot=Robot, location=Location)
visited = Fluent('visited', BoolType(), robot=Robot, location=Location)
connected = Fluent('connected', BoolType(), l_from=Location, l_to=Location)

In [3]:
move = InstantaneousAction('move', robot=Robot, l_from=Location, l_to=Location)
robot = move.parameter('robot')
l_from = move.parameter('l_from')
l_to = move.parameter('l_to')
move.add_precondition(at(robot, l_from))
move.add_precondition(connected(l_from, l_to))
move.add_effect(at(robot, l_from), False)
move.add_effect(at(robot, l_to), True)
move.add_effect(visited(robot, l_to), True)

In [4]:
r1 = Object('r1', Robot)
NLOC = 10
locations = [Object('l%s' % i, Location) for i in range(NLOC)]

problem = Problem('robot_with_simulated_effects')
problem.add_fluent(at, default_initial_value=False)
problem.add_fluent(visited, default_initial_value=False)
problem.add_fluent(connected, default_initial_value=False)
problem.add_action(move)

problem.add_object(r1)
problem.add_objects(locations)

problem.set_initial_value(at(r1, locations[0]), True)
problem.set_initial_value(visited(r1, locations[0]), True)
for i in range(NLOC - 1):
    problem.set_initial_value(connected(locations[i], locations[i+1]), True)
problem.set_initial_value(connected(locations[4], locations[8]), True)

In [5]:
goals = {}
goals[visited(r1, locations[5])] = -5
goals[visited(r1, locations[7])] = 4
goals[visited(r1, locations[9])] = 10
goals[visited(r1, locations[9])] = 20

problem.add_quality_metric(up.model.metrics.Oversubscription(goals))

In [6]:
with OneshotPlanner(name="symk") as planner:
    result = planner.solve(problem, output_stream=sys.stdout)
    plan = result.plan
    assert plan is not None
    print("%s returned: %s" % (planner.name, result.plan))

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_environment().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_39777/1572416823.py`, you are using the following planning engine:
  * Engine name: SymK
  * Developers:  David Speck (cf. https://github.com/speckdavid/symk/blob/master/README.md )
  * Description: SymK is a state-of-the-art domain-independent classical optimal and top-k planner.

[]
(define (domain robot_with_simulated_effects-domain)
 (:requirements :strips :typing)
 (:types robot location)
 (:predicates (at_ ?robot - robot ?location - location) (visited ?robot - robot ?location - location) (connected ?l_from - location ?l_to - location))
 (:action move
  :parameters ( ?robot - robot ?l_from - location ?l_to - location)
  :precondition (and (at_ ?robot ?l_from) (connected ?l_from ?l_to))
  :effect (and (not (at_ ?robot ?l_from)) (at_ ?robot ?l_to) (visited ?robot 

AssertionError: 

In [ ]:
# Validator
pv = PlanValidator(problem_kind=problem.kind)
pv_res = pv.validate(problem, result.plan)
pv_res.metric_evaluations[problem.quality_metrics[0]]